### Wrangling with missing or bogus data or any 0 values
- Either drop NaN data or fillin with reasonable value
- clean size information
- clean branding informaiton
- Calculate the wholesale price

### Import libraries and define universal variables

In [1]:
import numpy as np
import pandas as pd
import re
import math
import matplotlib.pyplot as plt
import codecs
from lib import utility as util
from functools import partial

### Import data
*** please don't proceec to the next cell before data tables is shown below the cell.***

In [2]:
# file paths
integrated_file = "processed_data/integrated.csv"
integrated_df = pd.read_csv( integrated_file, 
                             index_col='sales_date',
                             header=0,
                             parse_dates=['sales_date'],
                             low_memory=False )

sales_variables = ['retail_price','sales_proceeds','raw_cost','gross_profit','number_of_sales']

integrated_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 876339 entries, 2014-08-01 to 2017-07-28
Data columns (total 20 columns):
product_id             876339 non-null object
sales_id               876339 non-null int64
product_name           876339 non-null object
retail_price           876339 non-null int64
sales_proceeds         876339 non-null object
gross_profit           873639 non-null object
number_of_sales        876339 non-null int64
raw_cost               876339 non-null float64
manufacture_country    856216 non-null object
category               875465 non-null object
material_type          875160 non-null object
size                   844820 non-null object
quality                836162 non-null object
brand                  607592 non-null object
color                  594584 non-null object
subcategory_group      848428 non-null object
subcategory            848428 non-null object
size_letter            209937 non-null object
material_group         829492 non-null object
m

### Fixing data types

In [3]:
# convert all string value, which should be of float type, to float type
integrated_df[sales_variables] = integrated_df[sales_variables].applymap(util.convert_to_float)

# drop rows with invalid numeric data 
print("The dataframe length before invalid numerical data is dropped: {}".format(len(integrated_df)))
integrated_df = util.filter_by_invalid_numerics_in_df(integrated_df, sales_variables)
print("The dataframe length after invalid numerical data is dropped: {}".format(len(integrated_df)))

# convert data to originally intended data types
integrated_df[sales_variables] = integrated_df[sales_variables].astype(float)
integrated_df['number_of_sales'] = integrated_df['number_of_sales'].astype(int)
integrated_df['sales_id'] = integrated_df['sales_id'].astype(object)
print(integrated_df.info())

The dataframe length before invalid numerical data is dropped: 876339
The dataframe length after invalid numerical data is dropped: 876337
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 876337 entries, 2014-08-01 to 2017-07-28
Data columns (total 20 columns):
product_id             876337 non-null object
sales_id               876337 non-null object
product_name           876337 non-null object
retail_price           876337 non-null float64
sales_proceeds         876337 non-null float64
gross_profit           873637 non-null float64
number_of_sales        876337 non-null int64
raw_cost               876337 non-null float64
manufacture_country    856214 non-null object
category               875463 non-null object
material_type          875158 non-null object
size                   844818 non-null object
quality                836160 non-null object
brand                  607592 non-null object
color                  594582 non-null object
subcategory_group      848426 non-null ob

### Calculating gross profit and sold price

In [15]:
# possible whole sale price = (gross profit - ( raw_cost * # of sales)) / # of sales 
missing_gross_profits = integrated_df[integrated_df['gross_profit'].isnull()]
mraw_cost = missing_gross_profits['raw_cost'].apply(np.floor)
msales_proceeds = missing_gross_profits['sales_proceeds'].apply(np.floor)
mnumber_of_sales = missing_gross_profits['number_of_sales']
integrated_df.loc[integrated_df['gross_profit'].isnull(), 'gross_profit'] = msales_proceeds - (mraw_cost * mnumber_of_sales)

gross_profit = integrated_df['gross_profit'].apply(np.floor)
raw_cost = integrated_df['raw_cost'].apply(np.floor)
number_of_sales = integrated_df['number_of_sales']
integrated_df['sold_price'] = (gross_profit + (raw_cost * number_of_sales)) / number_of_sales
sales_variables.append('sold_price')

In [18]:
column_order = ['product_id',
                'sales_id',
                'product_name',
                'category',
                'subcategory_group',
                'subcategory',
                'color',
                'size_letter',
                'size',
                'brand',
                'manufacture_country',
                'material_type',
                'material_group',
                'material',
                'quality',
                'retail_price',
                'sold_price',
                'sales_proceeds',
                'number_of_sales',
                'raw_cost',
                'gross_profit'
               ]

integrated_df = integrated_df[column_order]
integrated_df.head(20)

,product_id,sales_id,product_name,category,subcategory_group,subcategory,color,size_letter,size,brand,...,material_type,material_group,material,quality,retail_price,sold_price,sales_proceeds,number_of_sales,raw_cost,gross_profit
sales_date,,,,,,,,,,,,,,,,,,,,,
2014-08-01,103591,20239746,アンティークシダー フォールディング テーブル （Ｌ）,インテリア雑貨,['テーブル'],['テーブル'],NaN,ｌ,８８×６０×７２ｃｍ,NaN,...,木製品,NaN,NaN,NaN,18000.0,9900.0,9900.0,1,3130.16,6770.0
2014-08-01,102862,20239747,アンティークシダー カードホルダー,文具・デスク用品,"['カード', 'ホルダー']","['カード', 'ホルダー']",NaN,NaN,１８．５×２４．５×８．５ｃｍ,NaN,...,木製品,['cedar'],['cedar'],（ＣＥＤＡＲ）,2200.0,1249.0,1210.0,1,507.62,742.0
2014-08-01,104782,20239747,ティーフフィル カード ラック,実用小物・消耗品,"['カード', 'ラック']","['カード', 'ラック']",NaN,NaN,４２×２０×３６ｃｍ,NaN,...,金属製品,['iron'],['iron'],ＩＲＯＮ,5500.0,3097.0,3025.0,1,1111.26,1986.0
2014-08-01,999000,20239747,立替 送料,その他,NaN,NaN,NaN,NaN,NaN,NaN,...,その他,NaN,NaN,NaN,0.0,600.0,600.0,1,0.00,600.0
2014-08-01,190038,20239751,ネックレス インターバル ヴァリエ ストーン,アクセサリー,['ネックレス'],['ネックレス'],NaN,NaN,４５ｃｍ,NaN,...,金属製品,"['アメジスト', 'goldplated', '藍晶石', 'オニキス', 'クォーツ',...","['amethyst', 'goldplated', 'labrado', 'onyx', ...",ＳＩＬＶＥＲ９２５（１６－Ｋ ＧＯＬＤ ＰＬＡＴＥＤ） ＰＵ：ＡＭＥＴＨＹＳＴ Ｌ・ＧＹ：Ｌ...,3800.0,2280.0,4560.0,2,984.00,2592.0
2014-08-01,190039,20239751,ブレスレット インターバル ヴァリエ ストーン,アクセサリー,"['ブレス', 'ブレス']","['ブレス', 'ブレスレット']",NaN,NaN,１８．５ｃｍ,NaN,...,金属製品,"['アメジスト', 'goldplated', '藍晶石', 'オニキス', 'クォーツ',...","['amethyst', 'goldplated', 'labrado', 'onyx', ...",ＳＩＬＶＥＲ９２５（１６－Ｋ ＧＯＬＤ ＰＬＡＴＥＤ） ＰＵ：ＡＭＥＴＨＹＳＴ Ｌ・ＧＹ：Ｌ...,3000.0,1800.0,3600.0,2,769.00,2062.0
2014-08-01,190044,20239751,ブレスレット アレンジ ウォーターパール,アクセサリー,"['ブレス', 'ブレス']","['ブレス', 'ブレスレット']",NaN,NaN,１８ｃｍ,NaN,...,金属製品,"['goldplated', 'pear', 'pearl', 'シルバー', 'パール']","['goldplated', 'pear', 'pearl', 'silver925', '...",ＳＩＬＶＥＲ９２５（１６－Ｋ ＧＯＬＤ ＰＬＡＴＥＤ） ＷＡＴＥＲ ＰＥＡＲＬ ＊ポーチ付,3000.0,1800.0,3600.0,2,688.00,2224.0
2014-08-01,190046,20239751,ネックレス ＢＲＡＳＳ ティップ,アクセサリー,['ネックレス'],['ネックレス'],NaN,NaN,５０ｃｍ,NaN,...,金属製品,"['brass', 'goldplated', 'シルバー']","['brass', 'goldplated', 'silver925']",ＳＩＬＶＥＲ９２５（１６－Ｋ ＧＯＬＤ ＰＬＡＴＥＤ） ＢＲＡＳＳ＝ＰＩＰＥ,4200.0,2520.0,5040.0,2,975.00,3090.0
2014-08-01,190047,20239751,ピアス ＢＲＡＳＳ ティップ,アクセサリー,['ピアス'],['ピアス'],NaN,NaN,１．５×４．５ｃｍ,NaN,...,金属製品,"['brass', 'goldplated', 'シルバー']","['brass', 'goldplated', 'silver925']",ＳＩＬＶＥＲ９２５（１６－Ｋ ＧＯＬＤ ＰＬＡＴＥＤ） ＢＲＡＳＳ＝ＰＩＰＥ,2800.0,1680.0,3360.0,2,574.00,2212.0


### Create a cleaned csv

In [19]:
integrated_df.to_csv('processed_data/cleaned_data.csv')